# Data Wrangling

A Python script that Audit and cleanse the dataset 'dataset1_with_error.csv', then output a clean data set 'dataset1_solution.csv'


Libraries used: 
* pandas 
* re 
* datetime
* collections
* difflib
* numpy

## Input:
    dataset1_with_error.csv

Different generic and major data problems could be found in the input data set:
- Typos and spelling mistakes
- Irregularities, e.g., abnormal data values and data formats
- Violations of the Integrity constraint.
- Outliers
- Duplications
- Missing values
- Inconsistency, e.g., inhomogeneity in values and types in representing the same data


## Output:
    'dataset1_solution.csv'


Format:

<style>
td {
  font-size: 80px
}
</style>

| COLUMN | [FORMAT] AND DESCRIPTION |
| --- | --- |
| Id | [Integer] 8 digit Id of the job advertisement |
| Title | [String] Title of the advertised job position |
| Location | [String] Location of the advertised job position |
| Company | [String] Company (employer) of the advertised job position |
| ContractType | [String] The contract type of the advertised job position, it could be full-time, part-time or non-specified |
| ContractTime | [String] The contract time of the advertised job position, it could be permanent, contract or non-specified |
| Category | [String] The Category of the advertised job position, e.g., IT jobs, Engineering Jobs, etc |
| Salary | [Float] Annual Salary of the advertised job position, e.g., 80000 |
| OpenDate | [String] The opening time for applying for the advertised job position, e.g.,20120104T150000, means 3pm, 4th January 2012 |
| CloseDate | [String] The closing time for applying for the advertised job position, e.g.,20120104T150000, means 3pm, 4th January 2012 |
| SourceName | [String] The website where the job position is advertised |


## Import libraries 

In [1]:
import pandas as pd
import re
import numpy as np
from collections import Counter
from difflib import SequenceMatcher
from datetime import datetime

## Task 1. Auditing and cleansing the loaded data

Here is the code for identify data problems and to fix them. 

## Part1. File reading and infomation gatherthing 

In [2]:
df = pd.read_csv('./dataset1_with_error.csv')

In [3]:
df.shape

(55169, 11)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55169 entries, 0 to 55168
Data columns (total 11 columns):
Id              55169 non-null int64
Title           55169 non-null object
Location        55169 non-null object
Company         51320 non-null object
ContractType    33493 non-null object
ContractTime    47047 non-null object
Category        55169 non-null object
Salary          53584 non-null object
OpenDate        55169 non-null object
CloseDate       55169 non-null object
Source          55169 non-null object
dtypes: int64(1), object(10)
memory usage: 4.6+ MB


In [5]:
# Describing the Id would not make any scence so we only do it on object type items
df.describe(include=['O'])

,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,Source
count,55169,55169,51320,33493,47047,55169,53584,55169,55169,55169
unique,55166,489,9064,4,4,8,3757,2194,2418,106
top,Pensions Administrators (Temporary/Contract),UK,,,permanent,IT Jobs,35000,20120415T150000,20131208T000000,totaljobs.com
freq,2,8397,1133,14902,33637,14344,1865,45,45,10102


In [6]:
df.isnull().sum()

Id                  0
Title               0
Location            0
Company          3849
ContractType    21676
ContractTime     8122
Category            0
Salary           1585
OpenDate            0
CloseDate           0
Source              0
dtype: int64

In [7]:
df.head(3)

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,Source
0,12612628,Engineering Systems Analyst,Dorking,Gregory Martin International,NaN,permanent,Engineering Jobs,25000,20130708T120000,20130906T120000,cv-library.co.uk
1,12612830,Stress Engineer Glasgow,Glasgow,Gregory Martin International,NaN,permanent,Engineering Jobs,30000,20120130T000000,20120330T000000,cv-library.co.uk
2,12612844,Modelling and simulation analyst,Hampshire,Gregory Martin International,NaN,permanent,Engineering Jobs,30000,20121221T150000,20130120T150000,cv-library.co.uk


After the study of the data, we found that 'ContractType', 'ContractTime' and 'Category' are easy to clean if we compare these to 'Salary' or 'Company' column. Hence we will focus on these columns first.

## Part2. 'Contract' and 'Category'

    Part2.a
    ContractTime and ContractType

In [8]:
df.ContractTime.value_counts()

permanent    33637
-             6249
contract      6089
              1072
Name: ContractTime, dtype: int64

In [9]:
# All we need to do is keeping 'permanent' and 'contract', then replace NaN, '-' and space with 'non-specified' as required
df.ContractTime.fillna("non-specified", inplace = True) 
df.ContractTime=df.ContractTime.replace(r'(^\s+$|^-$)', 'non-specified', regex=True)

In [10]:
print (df.ContractTime.value_counts())

permanent        33637
non-specified    15443
contract          6089
Name: ContractTime, dtype: int64


In [11]:
# Then in the same way, for 'ContractType'
df.ContractType.value_counts()

             14902
full_time    12303
-             4720
part_time     1568
Name: ContractType, dtype: int64

In [12]:
# Similarly, the output in this column should be 'full-time','part-time' and 'no-specified'
df.ContractType.fillna("non-specified", inplace = True) 
df.ContractType=df.ContractType.replace(r'(^\s+$|^-$)', 'non-specified', regex=True)
df.ContractType=df.ContractType.replace(r'_', '-', regex=True)

In [13]:
print (df.ContractType.value_counts())

non-specified    41298
full-time        12303
part-time         1568
Name: ContractType, dtype: int64


    Part2.b
    What inside the 'Category'?

In [14]:
print (df.Category.value_counts())

IT Jobs                             14344
Healthcare & Nursing Jobs            8809
Engineering Jobs                     8210
Accounting & Finance Jobs            7138
Sales Jobs                           5349
Hospitality & Catering Jobs          4788
Teaching Jobs                        3779
PR, Advertising & Marketing Jobs     2752
Name: Category, dtype: int64


Seems the 'Category' is meet our requirement and display without the NaN value inside. So we can just keep it.
let us see the data frame looks so far.

In [15]:
df.describe(include=['O'])

,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,Source
count,55169,55169,51320,55169,55169,55169,53584,55169,55169,55169
unique,55166,489,9064,3,3,8,3757,2194,2418,106
top,Pensions Administrators (Temporary/Contract),UK,,non-specified,permanent,IT Jobs,35000,20120415T150000,20131208T000000,totaljobs.com
freq,2,8397,1133,41298,33637,14344,1865,45,45,10102


## Part3. We now move to 'Location', 'Company' and 'Source'

     Part3.a 
     We first focus on the 'Company' colume

In [16]:
print (df.Company.value_counts())

                                    1133
UKStaffsearch                        498
-                                    406
Randstad                             402
CVbrowser                            328
Penguin Recruitment                  252
Matchtech Group plc.                 242
London4Jobs                          201
COREcruitment International          192
Hays                                 190
JOBG8                                183
Chef Results                         177
JAM Recruitment Ltd                  150
Populus Healthcare                   146
Michael Page Finance                 136
Computer People                      135
Senitor Associates                   132
Bee Recruitment London Ltd           124
Evolution Recruitment Solutions      111
Liquid Personnel Ltd                 109
Experis IT                           109
SYK Recruitment                      107
Computer Futures                     107
Adecco                               106
Clear Selection 

In [17]:
# there are some missing values, typo and Inconsistency.
# We need introduce different symbols to represent missingness later on for Company from different source 
print ("number of missing:",df.Company.isnull().sum())

number of missing: 3849


In [18]:
# the following function is the clean and normalize the company field.
def removeEndSpecialCharCompany(company):
    if pd.isnull(company):
        return company
    else:
        # normalize to upper case letters
        company = company.upper()
        # remove special character at the end
        company = re.sub(r"\W+$", "", company)
        # replace LIMITED with LTD
        company = company.replace("LIMITED", "LTD")
        # replace " AND " with " & " for processing 
        company = company.replace(" & ", " AND ")
        # remove all special characters except space and dot
        company = re.sub(r'([^\w\s\.]|_)', '', company)
        # change " AND " back to " & " 
        company = company.replace(" AND ", " & ")
        # replace multiple spaces with a single space, also trim spaces on both side
        company = re.sub( '\s+', ' ', company).strip()
        return company
print (len(set(df.Company)), "different companies in the current data set")
df['Company'] = df.Company.apply(lambda x: removeEndSpecialCharCompany(x))
print (len(set(df.Company)), "different companies after normalization")

9065 different companies in the current data set
8544 different companies after normalization


In [19]:
#check the similarity between two string
# This function is quoted from 'data cleansing.ipynb'
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [20]:
# find the high frequency location to low frequency location
def match_highfreq_To_lowfreq(data, min_threshold, max_threshold):
    # check low frequency company names
    lowfreq_data_list = [k for k,v in Counter(data).items() if v<=5]
    print ("Number of lowfreq_data", len(lowfreq_data_list))
    highfreq_data_list = list(set(data[~(data.isnull())]).difference(lowfreq_data_list))
    print ("Number of highfreq_data", len(highfreq_data_list))
    # the printed data tells that probably not many companies posted more than 5 jobs in those years.


    match_data={}
    for lowfreq_data in lowfreq_data_list:
        for highfreq_data in highfreq_data_list:
            ratio = similar(lowfreq_data.upper(), highfreq_data.upper())
            if ratio >= min_threshold and ratio <= max_threshold:
                match_data[lowfreq_data] = highfreq_data
                print(lowfreq_data, ":", highfreq_data, ratio)
    return match_data

In [21]:
# Apply the function and find  the company with typo
match_data_company=match_highfreq_To_lowfreq(df.Company, 0.95, 1)

Number of lowfreq_data 6600
Number of highfreq_data 1943
INSPIRE PEOPLE : INSPIRED PEOPLE 0.9655172413793104
RANDSTAD FINANCIAL PROFESSIONAL : RANDSTAD FINANCIAL & PROFESSIONAL 0.96875
JANE LEWIS HEALTH & SOCIAL CARE : JANE LEWIS HEALTH SOCIAL CARE 0.9666666666666667
WEBRECRUIT LTD : WEB RECRUIT LTD 0.9655172413793104
SIGMAR RECRUITMENT : SIGMA RECRUITMENT 0.9714285714285714
SOLUTION RECRUITMENT : SOLUTIONS RECRUITMENT 0.975609756097561
CCITY IT RECRUITMENT : C CITY IT RECRUITMENT 0.975609756097561
PROSPER RECRUITMENT : PROSPERO RECRUITMENT 0.9743589743589743
TRAVAIL EMPLOYMNET GROUP : TRAVAIL EMPLOYMENT GROUP 0.9583333333333334
ADMIRAL HOTELS RESTAURANTS : ADMIRAL HOTELS & RESTAURANTS 0.9629629629629629
BLUETOWNONLINE : BLUETOWN ONLINE 0.9655172413793104
PRIMETIME RECRUITMENT : PRIME TIME RECRUITMENT 0.9767441860465116
EDSTAFF LTD : EDUSTAFF LTD 0.9565217391304348
SOLUTIONS2RECRUITMENT : SOLUTIONS RECRUITMENT 0.9523809523809523
BERKELEY SCOTT CONTRACT CATERING FACILITIES MANAGEMENT : 

In [22]:
# replace lower frequent with higher frequent company name
# also replace NaN and space with 'NON-SPECIFIED'
df.Company=df.Company.replace(match_data_company)
df.Company=df.Company.replace(r'^\s*$', 'NON-SPECIFIED', regex=True)
df.Company.fillna('NON-SPECIFIED', inplace = True)

In [23]:
# Check the company again
print (df.Company.value_counts())

NON-SPECIFIED                                   5389
UKSTAFFSEARCH                                    498
RANDSTAD                                         403
CVBROWSER                                        328
MATCHTECH GROUP PLC                              286
PENGUIN RECRUITMENT                              252
HAYS                                             230
LONDON4JOBS                                      201
CORECRUITMENT INTERNATIONAL                      197
JAM RECRUITMENT LTD                              192
JOBG8                                            184
CHEF RESULTS                                     177
POPULUS HEALTHCARE                               146
ADDITIONAL RESOURCES LTD                         142
MICHAEL PAGE FINANCE                             136
COMPUTER PEOPLE                                  135
SENITOR ASSOCIATES                               132
IDEX CONSULTING LLP                              131
BEE RECRUITMENT LONDON LTD                    

     Part3.b
     Now we look at 'Location'

In [24]:
print (df.Location.value_counts())

UK                      8397
London                  7046
South East London       2961
The City                1184
Central London           889
Manchester               754
Leeds                    733
Surrey                   686
Birmingham               639
Reading                  604
West Midlands            579
Bristol                  471
Berkshire                441
East Sheen               437
Nottingham               409
Oxford                   395
Hampshire                387
Milton Keynes            383
Sheffield                383
Kent                     362
Cambridge                358
Newcastle Upon Tyne      346
Guildford                340
Leicester                311
Hertfordshire            301
Liverpool                286
Essex                    282
Aberdeen                 282
Oxfordshire              275
West Yorkshire           273
                        ... 
Twickenham                11
Lewisham                  11
Lutterworth               11
Oldbury       

In [25]:
# Replace any '-' with one space to keep the consistency
df.Location = df.Location.replace('-', ' ',regex=True)

In [26]:
# We apply the match funtion and set the min_threshold to 0.8 due to the shorter string lenght of the location
match_data_location=match_highfreq_To_lowfreq(df.Location, 0.80, 1)

Number of lowfreq_data 10
Number of highfreq_data 479
Leads : Leeds 0.8
london : London 1.0
Livepool : Liverpool 0.9411764705882353
Oxfords : Oxford 0.9230769230769231
HAMpshire : Hampshire 1.0
Manchaster : Manchester 0.9
Manchaster : Lancaster 0.8421052631578947
birmingham : Birmingham 1.0
ABERDEEN : Aberdeen 1.0
BRISTOL : Bristol 1.0
Cembridge : Cambridge 0.8888888888888888


In [27]:
# After check the result, we decided to replace the lower frequency word with higher one
df.Location=df.Location.replace(match_data_location)

In [28]:
print (df.Location.value_counts())

UK                      8397
London                  7048
South East London       2961
The City                1184
Central London           889
Manchester               754
Leeds                    734
Surrey                   686
Birmingham               641
Reading                  604
West Midlands            579
Bristol                  472
Berkshire                441
East Sheen               437
Nottingham               409
Oxford                   397
Hampshire                389
Sheffield                383
Milton Keynes            383
Kent                     362
Cambridge                359
Newcastle Upon Tyne      346
Guildford                340
Leicester                311
Hertfordshire            301
Liverpool                287
Aberdeen                 284
Essex                    282
Oxfordshire              275
West Yorkshire           273
                        ... 
Knaresborough             12
Broadfield                11
Normanton                 11
Dumfries      

In [29]:
# 'City' and 'The City' sounds the same, 'City' only appears 10 times so we can replace it with 'The City'
df.loc[df['Location'] == 'City', 'Location'] = 'The City'

In [30]:
print (df.Location.value_counts())

UK                      8397
London                  7048
South East London       2961
The City                1194
Central London           889
Manchester               754
Leeds                    734
Surrey                   686
Birmingham               641
Reading                  604
West Midlands            579
Bristol                  472
Berkshire                441
East Sheen               437
Nottingham               409
Oxford                   397
Hampshire                389
Sheffield                383
Milton Keynes            383
Kent                     362
Cambridge                359
Newcastle Upon Tyne      346
Guildford                340
Leicester                311
Hertfordshire            301
Liverpool                287
Aberdeen                 284
Essex                    282
Oxfordshire              275
West Yorkshire           273
                        ... 
Frome                     12
Knaresborough             12
Broadfield                11
Normanton     

    Part3.c
    We then look at column 'Source'

In [31]:
print (df.Source.value_counts())

totaljobs.com                     10102
cv-library.co.uk                   7841
jobsite.co.uk                      3630
cwjobs.co.uk                       3132
staffnurse.com                     2778
caterer.com                        2335
Jobcentre Plus                     2143
careworx.co.uk                     1966
jobserve.com                       1718
jobs.catererandhotelkeeper.com     1250
jobs.guardian.co.uk                1182
MyUkJobs                           1106
fish4.co.uk                        1074
theitjobboard.co.uk                 814
rengineeringjobs.com                717
gojobsearch.co.uk                   683
salestarget.co.uk                   637
planetrecruit.com                   628
GAAPweb                             599
ukstaffsearch.com                   499
technojobs.co.uk                    470
strike-jobs.co.uk                   461
Brand Republic Jobs                 426
justengineers.net                   419
eFinancialCareers                   389


In [32]:
# find the Source that not end with .com, .uk, .net and .org
website_df = df[~df["Source"].str.contains('.com$|.uk$|.net$|.org$')]

In [33]:
website_df.Source.value_counts()

Jobcentre Plus                 2143
MyUkJobs                       1106
GAAPweb                         599
Brand Republic Jobs             426
eFinancialCareers               389
PR Week Jobs                    178
Multilingualvacancies           119
Jobs Ac                          97
Jobs24                           86
ijobs                            75
JobSearch                        37
JustLondonJobs                   22
Teaching jobs - TES Connect      16
TotallyExec                       7
Name: Source, dtype: int64

In [34]:
# create two lists so we can replace webname with real website address
list_a=list(website_df.Source.value_counts().keys())

In [35]:
list_b=['gov.uk','myukjobs.co.uk','gaapweb.com','campaignlive.co.uk','efinancialcareers.co.uk','prweekjobs.co.uk',
       'multilingualvacancies.com','jobs.ac.uk','jobs24.co.uk','ijobs.com.uk','jobsearch.co.uk','justlondonjobs.co.uk','tes.com',
        'totallyexec.com']

In [36]:
# replace the website name with real website address
df.Source.replace(list_a, list_b, inplace=True)

In [37]:
# Check it, is there any typo or similar website address?
match_data_source=match_highfreq_To_lowfreq(df.Source, 0.80, 1)

Number of lowfreq_data 6
Number of highfreq_data 100


In [38]:
# take a look again
df.Source.value_counts()

totaljobs.com                     10102
cv-library.co.uk                   7841
jobsite.co.uk                      3630
cwjobs.co.uk                       3132
staffnurse.com                     2778
caterer.com                        2335
gov.uk                             2143
careworx.co.uk                     1966
jobserve.com                       1718
jobs.catererandhotelkeeper.com     1250
jobs.guardian.co.uk                1182
myukjobs.co.uk                     1106
fish4.co.uk                        1074
theitjobboard.co.uk                 814
rengineeringjobs.com                717
gojobsearch.co.uk                   683
salestarget.co.uk                   637
planetrecruit.com                   628
gaapweb.com                         599
ukstaffsearch.com                   499
technojobs.co.uk                    470
strike-jobs.co.uk                   461
campaignlive.co.uk                  426
justengineers.net                   419
efinancialcareers.co.uk             389


## Part4.  'Id' column and 'Title' column

    Part4.a
    Id column seems easy and it will be the primary key in database

In [39]:
# Change Id type to string
df['Id'] = df['Id'].apply(str)

In [40]:
# Is there any Id in different format?
df[~df["Id"].str.match('^\d{8}$')]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,Source


In [41]:
# Is the Id unique?
df['Id'].is_unique

True

In [42]:
# Is the Id in increasing order?
df.Id.is_monotonic_increasing

True

In [43]:
# Change back to int, Id all good
df['Id'] = df['Id'].apply(int)

    Part4. b
    Title seems vary hard to deal with, because I lack of the idea and skill to build a methodology for complex nature language processing. The ideal result in this column will be the primary job title without location, salary and useless descriptions. The formation should be consistent, for example, only use &, / and , in proper location.

In [44]:
print (df.Title.value_counts())

Pensions Administrators (Temporary/Contract)                               2
Quality Assurance Environmental Manager Nottingham, East Midlands          2
Senior Financial Advisor                                                   2
Senior SEO Specialist On/Offpage / Google Analytics Harrogate              1
Field Sales  Document Scanning / Bureau  Coventry  ****k OTE               1
Purchaser  Joinery Experience Required                                     1
Lead Nurse (RGN), Diabetes Specialist (Telehealth)  Watford  Watford       1
Diary & Events Coordinator  De Vere Venues Ltd ?? Colmore Gate             1
Social Worker Family Centre                                                1
Business Development : Alternative Investment                              1
Internal Account Manager  ******** Bounds Green                            1
Compensation Specialist                                                    1
Investment Analyst  Fund of Funds Research                                 1

In [45]:
df.isnull().sum()

Id                 0
Title              0
Location           0
Company            0
ContractType       0
ContractTime       0
Category           0
Salary          1585
OpenDate           0
CloseDate          0
Source             0
dtype: int64

In [46]:
# the following function is the clean and normalize the Title field.
def removeEndSpecialCharTitle(title):
    # normalize to upper case letters
    title = title.upper()
    # replace LIMITED with LTD
    title = title.replace("LIMITED", "LTD")
    # deal with special characters
    title = title.replace("|", ' / ')
    title = re.sub(r'\s*\(.+\)','', title)
    title = title.replace(';|:',' , ')
    title = title.replace('"','')
    # deal with '****K' type of string
    title = re.sub('\*+K?\**[\*|K]$','',title)
    title = title.replace('*','')
    # remove the salary, for example 30,000
    title = re.sub('\d+,\d+','',title)
    # Deal with redundant 'LLOCATION' and 'SALARY'
    title = title.replace('LOCATION', ' ')
    title = title.replace('SALARY',' ')
    # remove all special characters except space, dot, ',' '\' and '+'
    title = re.sub(r'([^\w\s\.\,\/\+])', '', title)
    # replace multiple spaces with a single space, also trim spaces on both side
    title = re.sub( '\s+', ' ', title).strip()
    return title

print (len(set(df.Title)), "different titles in the current data set")
df['Title'] = df.Title.apply(lambda x: removeEndSpecialCharTitle(x))
print (len(set(df.Title)), "different titles after normalization")

55166 different titles in the current data set
53218 different titles after normalization


In [47]:
print (df.Title.value_counts())

PROJECT MANAGER                                                                  55
WEB DEVELOPER                                                                    34
SOFTWARE DEVELOPER                                                               31
MAINTENANCE ENGINEER                                                             26
FIELD SERVICE ENGINEER                                                           24
BUSINESS ANALYST                                                                 24
MECHANICAL DESIGN ENGINEER                                                       21
DESIGN ENGINEER                                                                  20
ACCOUNT MANAGER                                                                  19
SERVICE ENGINEER                                                                 18
BUSINESS DEVELOPMENT MANAGER                                                     17
PROJECT ENGINEER                                                            

In [48]:
# Apply the match data function and return the dict for high and low frequency data
match_data_title=match_highfreq_To_lowfreq(df.Title, 0.95, 1)

Number of lowfreq_data 53149
Number of highfreq_data 69
ACCOUNT MANGER : ACCOUNT MANAGER 0.9655172413793104
SUPPORT WORKERS : SUPPORT WORKER 0.9655172413793104
C SENIOR SOFTWARE ENGINEER : SENIOR SOFTWARE ENGINEER 0.96
MECHANICAL DESIGN ENGEER : MECHANICAL DESIGN ENGINEER 0.96
WEB DEVELOPERS : WEB DEVELOPER 0.9629629629629629
SENIOR SOFTWARE ENGINEER X : SENIOR SOFTWARE ENGINEER 0.96
ACCOUNTS MANAGER : ACCOUNT MANAGER 0.967741935483871
ACCOUT MANAGER : ACCOUNT MANAGER 0.9655172413793104
MECHANCICAL DESIGN ENGINEER : MECHANICAL DESIGN ENGINEER 0.9811320754716981
SALES EXECUTIVES : SALES EXECUTIVE 0.967741935483871
BUSINESS DEVELOPMENT MANGER : BUSINESS DEVELOPMENT MANAGER 0.9818181818181818
PROJECT ENGINEERS : PROJECT ENGINEER 0.9696969696969697
FIELD SALES EXECUTIVES : FIELD SALES EXECUTIVE 0.9767441860465116
MECHANICAL FITTERS : MECHANICAL FITTER 0.9714285714285714
FIELD SERVICE ENGINEERS : FIELD SERVICE ENGINEER 0.9777777777777777
CNET DEVELOPER : C NET DEVELOPER 0.9655172413793104
C

In [49]:
# replace the low frequency job title with high one
df.Title=df.Title.replace(match_data_title)

df.isnull().sum()

Id                 0
Title              0
Location           0
Company            0
ContractType       0
ContractTime       0
Category           0
Salary          1585
OpenDate           0
CloseDate          0
Source             0
dtype: int64

In [50]:
# This is the only Title that with '****', we need to replace it with 'NON-SPECIFIED'
df.loc[df.Id==50870368,'Title'] = 'NON-SPECIFIED'

In [51]:
print (df.Title.value_counts())

PROJECT MANAGER                                                                  60
WEB DEVELOPER                                                                    37
SOFTWARE DEVELOPER                                                               35
FIELD SERVICE ENGINEER                                                           29
MAINTENANCE ENGINEER                                                             26
BUSINESS ANALYST                                                                 26
BUSINESS DEVELOPMENT MANAGER                                                     25
MECHANICAL DESIGN ENGINEER                                                       25
ACCOUNT MANAGER                                                                  23
DESIGN ENGINEER                                                                  21
SERVICE ENGINEER                                                                 18
SOFTWARE ENGINEER                                                           

## Part5. Salary normalisation and imputation

Frist we focus on the normalisation part

In [52]:
df.Salary.value_counts()

35000         1865
30000         1827
40000         1694
32500         1461
45000         1437
37500         1361
25000         1297
50000         1218
27500         1143
42500         1067
55000          924
20000          853
24000          847
22500          804
0              773
47500          753
60000          734
18000          626
52500          582
26000          574
19000          566
22000          564
21000          553
29000          553
28000          541
23000          502
27000          490
26500          427
21500          427
17000          426
              ... 
25386            1
45024            1
25555            1
19000 pa         1
33500 pa         1
22192            1
39952            1
29307            1
77001            1
85800            1
44296.0          1
37680            1
23744            1
49125            1
50462            1
35942            1
22924            1
47300            1
20175            1
39811            1
33151            1
22867       

After invesgation, we found this column has several issues:
- vaild salary number or float number -> can be convert to float dirctly
- Nan, 0 or other characters that do not represent a vaild salary -> need to do the imputation
- vaild salary with extra characters, include 'per year','annum','p/h','pa' and so on -> need to remove those characters then convert the number in annual salary
- vaild salary in range and split by '-' and 'to'-> need to take the number in order to calculate the average salary
- not vaild salary with 'k' in the end -> need to cut 'k' then times 1000 to get the annual salary
- other suspicious numbers, for example a four digits number may repersent the monthly wage, need to be considered further
- extremely high Salary (greater than 10000) -> need to check the row to decide whether they are legit or not

I do not include the process of detecting these issues to keep the code tidy.

In [53]:
# replace NaN with '0'
df.Salary.fillna('0', inplace = True)
# replace '-' with '0'
df.Salary.replace('-','0',inplace= True)
# set all the words to lowercase
df.Salary=df.Salary.str.lower()

# define a functin to check the pattern of Salary number column
# We know if a salary number is  5 digits or float number with 5 digits before the ., it is probably valid 
def check_salary(value, pattern=re.compile('^\d{5}$|^\d{5}\.\d$')):
    return 'OK' if pattern.match(value) else 'Failed'

# Checking the salary column and create a new columne named 'test_salary' 
df['test_salary'] = df['Salary'].apply(check_salary)
print(len(df[df['test_salary']!='OK']))
df[df['test_salary']!='OK']# return nothing only when all rows are consistent with the pattern observed

3527


,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,Source,test_salary
17,25337172,PASTRY CHEF FOR RED STAR ROSETTE HOTEL,North East England,CLEAR SELECTION,non-specified,non-specified,Hospitality & Catering Jobs,0,20121214T120000,20121228T120000,caterer.com,Failed
21,25892031,MICE SALES AND MARKETING MANAGER,Buckinghamshire,CHESS PARTNERSHIP,non-specified,non-specified,Hospitality & Catering Jobs,0,20130603T000000,20130703T000000,caterer.com,Failed
28,27754996,QA ENGINEER,Cambridge,INDIGO 21 LTD,non-specified,permanent,IT Jobs,0,20130810T150000,20131108T150000,cv-library.co.uk,Failed
53,34558721,CHEF DE PARTIE BURTON ON TRENT UP TO,Staffordshire,CLEAR SELECTION,non-specified,non-specified,Hospitality & Catering Jobs,0,20131229T150000,20140112T150000,caterer.com,Failed
71,38909389,CHEF DE PARTIE FINE DINING RESTAURANT STUNNING...,Derbyshire,CHEF RESULTS,non-specified,non-specified,Hospitality & Catering Jobs,0,20120522T000000,20120721T000000,caterer.com,Failed
100,46126154,NEWLY QUALIFIED ROTATION SCHEME,London,CARNEGIE CONSULTING,non-specified,non-specified,Accounting & Finance Jobs,0,20120724T150000,20120807T150000,efinancialcareers.co.uk,Failed
120,46626571,RGN REGISTERED NURSE JOB DURHAM,UK,NON-SPECIFIED,full-time,non-specified,Healthcare & Nursing Jobs,0,20131025T000000,20131224T000000,careworx.co.uk,Failed
156,46627119,QUALIFIED SOCIAL WORKER JOB LONDON,North London,NON-SPECIFIED,full-time,non-specified,Healthcare & Nursing Jobs,0,20120420T120000,20120719T120000,careworx.co.uk,Failed
184,46627361,PAEDIATRIC STAFF NURSE JOB LONDON,London,NON-SPECIFIED,full-time,non-specified,Healthcare & Nursing Jobs,0,20130606T000000,20130706T000000,careworx.co.uk,Failed
234,46627890,HEALTH CARE ASSISTANT JOB SURREY,Surrey,NON-SPECIFIED,non-specified,non-specified,Healthcare & Nursing Jobs,0,20130105T120000,20130204T120000,careworx.co.uk,Failed


Inside 3527 rows, we can deal with '0' later by imputation. Right now we need to pick those non-zero rows to work with

In [54]:
# We have 796 raws to work with. We need to figure it out the different types of problem
df[(df['Salary']!='0') & (df['test_salary']!='OK')]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,Source,test_salary
418,46631494,UNIT MANAGER STAFFORDSHIRE,Staffordshire,NON-SPECIFIED,full-time,non-specified,Healthcare & Nursing Jobs,30250/year,20130717T150000,20130915T150000,careworx.co.uk,Failed
456,46632299,SEXUAL HEALTH CLINIC NURSE JOB ESSEX,Essex,NON-SPECIFIED,full-time,non-specified,Healthcare & Nursing Jobs,31500 per annum,20121118T120000,20130216T120000,careworx.co.uk,Failed
614,46634425,CHARGE NURSE RMN/RNLD KENT,Kent,NON-SPECIFIED,full-time,non-specified,Healthcare & Nursing Jobs,30500 - 35500,20131106T150000,20131206T150000,careworx.co.uk,Failed
650,46634927,RGN NIGHT NURSE DEWSBURY P/H,Dewsbury,NON-SPECIFIED,full-time,non-specified,Healthcare & Nursing Jobs,26500/year,20120920T000000,20121119T000000,careworx.co.uk,Failed
713,46635738,SENIOR STAFF NURSE ITU,The City,NON-SPECIFIED,non-specified,non-specified,Healthcare & Nursing Jobs,33000/year,20130326T000000,20130425T000000,careworx.co.uk,Failed
727,46635911,RGN STAFF NURSES FOR NURSING HOME SOUTHALL,UK,NON-SPECIFIED,non-specified,non-specified,Healthcare & Nursing Jobs,23952/year,20121002T120000,20121201T120000,careworx.co.uk,Failed
852,46852076,RMN/RGN NURSE STOKE,UK,NON-SPECIFIED,non-specified,non-specified,Healthcare & Nursing Jobs,26131 to 28131,20120715T000000,20120913T000000,careworx.co.uk,Failed
909,48271564,RMN,Oxford,NON-SPECIFIED,non-specified,non-specified,Healthcare & Nursing Jobs,24646/year,20130822T000000,20131021T000000,careworx.co.uk,Failed
1101,51593378,HOME CARE WORKERS WATFORD,Watford,NON-SPECIFIED,part-time,non-specified,Healthcare & Nursing Jobs,8160,20120112T120000,20120211T120000,careworx.co.uk,Failed
1131,52269379,MULTI SITE PRACTICE MANAGER BERKSHIRE/LONDON,London,NON-SPECIFIED,non-specified,non-specified,Healthcare & Nursing Jobs,31000/year,20121016T120000,20121030T120000,careworx.co.uk,Failed


In [55]:
# Take a look at the salary that less than 10000. We found that it may due to the apprentice job title or part-time contract type.
# Hence we keep the data for further analysis.
df[(df['Salary'].str.match('^\d{4}$|^\d{4}\.')) & (df['test_salary']!='OK')]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,Source,test_salary
1101,51593378,HOME CARE WORKERS WATFORD,Watford,NON-SPECIFIED,part-time,non-specified,Healthcare & Nursing Jobs,8160,20120112T120000,20120211T120000,careworx.co.uk,Failed
1210,53652065,PART TIME WEEKEND CARERS POSITIONS AVAILABLE P...,UK,NON-SPECIFIED,part-time,non-specified,Healthcare & Nursing Jobs,7680,20131029T120000,20131112T120000,careworx.co.uk,Failed
1749,57752163,PR NEW TEST,Cambridgeshire,OCC COMPUTER PERSONNEL LTD,full-time,permanent,IT Jobs,7200,20130502T000000,20130731T000000,planetrecruit.com,Failed
1951,59788339,"FIELD CARE SUPERVISOR, WINCHESTER",Winchester,NON-SPECIFIED,part-time,non-specified,Healthcare & Nursing Jobs,7680,20121120T150000,20121204T150000,careworx.co.uk,Failed
2255,62004217,TELESALES APPRENTICE,Doncaster,OPTIMAS TRAINING,non-specified,non-specified,IT Jobs,5184,20120104T150000,20120118T150000,myukjobs.co.uk,Failed
2351,62006760,APPRENTICE SALES MARKETING EXECUTIVE,Lutterworth,OPTIMAS TRAINING,non-specified,non-specified,"PR, Advertising & Marketing Jobs",6000,20120820T000000,20121019T000000,myukjobs.co.uk,Failed
2398,62007710,CALL CENTRE APPRENTICE,Manchester,RECRUITMENT NORTH WEST,non-specified,non-specified,Accounting & Finance Jobs,5760,20130225T000000,20130311T000000,myukjobs.co.uk,Failed
2436,62009519,CNC MACHINIST BORER,Glenrothes,CCN RECRUITMENT,non-specified,non-specified,Engineering Jobs,5268,20120906T000000,20121105T000000,myukjobs.co.uk,Failed
4005,65688880,PART TIME TEL ACCOUNT MANAGER,Exeter,SPRING PERSONNEL,non-specified,permanent,Sales Jobs,9975,20131010T120000,20140108T120000,cv-library.co.uk,Failed
4654,66424531,COMPLIANCE ADMINISTRATOR SHREWSBURY,Shrewsbury,NON-SPECIFIED,part-time,non-specified,Healthcare & Nursing Jobs,7500,20130428T000000,20130512T000000,careworx.co.uk,Failed


4.a we take look for these salary with a range and split by '-' or 'to'. We need to calculate the average salary base on the data. Then assign it back to df

In [56]:
# create the mask so we can filte the data from dataframe
mask_a=(df['Salary'].str.match('^\d{5}\s(-|to)\s\d{5}$')) & (df['test_salary']!='OK')

In [57]:
# calculate the average salary based on the range
cal_a=(df.Salary[mask_a].str.split(' to | - ', n=1).str[0].astype(int) + df.Salary[mask_a].str.split(' to | - ', n=1).str[1].astype(int))/2

In [58]:
# assign it back to dataframe
df.loc[mask_a, 'Salary']=cal_a

4.b We then look at the salary with 'p/h' and 'hour'. We found that the format looks like \d{2} (p/h|hour). So we need to cut the words then calculate the salary for whole year. I checked the relative website that typical british worker may work around 30-36 hours per week. Since the job type is 'part-time' inside this mask, I use 30hour/week.

In [59]:
# create the mask for salary in hour
mask_b=(df['Salary'].str.contains('p/h|hour') & (df['test_salary']!='OK'))

In [60]:
# calculate the result then sign it back
df.loc[mask_b, 'Salary']=df.Salary[mask_b].str.findall('(\d{2})').str.get(0).astype(float)*30*52

4.c Next, we look at the valid salary with extra words in the end. We need cut those words

In [61]:
#  create the mask to filter the group
mask_c=((df['Salary'].str.contains('year|per|pa|annum')) & (df['Salary'].str.contains('^\d{5}')))& (df['test_salary']!='OK')
#  cut the words at the end then assign it back
df.loc[mask_c, 'Salary']=df.Salary[mask_c].str.findall('\d{5}').str.get(0)

4.d Finally, we look at the word 'k' situation. We need cut 'k' first then times 1000 to get the salary

In [62]:
# create mask and do the string adding
mask_d=(df['Salary'].str.contains('k') & df.Salary.str.contains('\d{2}')) & (df['test_salary']!='OK')
df.loc[mask_d, 'Salary']=df.Salary[mask_d].str.findall('(\d{2})').str.get(0)+'000'

In [63]:
# assign the data type to string
df.Salary=df['Salary'].astype(str)

So far we have solve four kinds of major issue in Salary column. Are there more issues?

In [64]:
# check the issue, is there any?
df[~(df['Salary'].str.match('^\d{5}$|^\d{4}$|^\d{5}\.\d$|^\d{4}\.\d$|0'))]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,Source,test_salary
2185,61822755,SHAREPOINT SOLUTION ANALYST / DEVELOPER,Edinburgh Technopole,365 LTD,non-specified,permanent,IT Jobs,120000,20130505T150000,20130803T150000,cwjobs.co.uk,Failed
15183,68495591,IOS DEVELOPER PERMANENT CENTRAL LONDON,London,GREGORY JAMES,non-specified,permanent,IT Jobs,150000,20120423T150000,20120722T150000,jobserve.com,Failed
26971,69181134,OFFICE TECHNICAL ADMINISTRATOR,Cheshire,GROUP CYTEK,full-time,permanent,Engineering Jobs,10000000,20130325T120000,20130424T120000,workthing.com,Failed
48506,71869302,NEW BUSINESS DEVELOPMENT EXEC/MANAGER,Central London,PERISCOPE UK LTD,full-time,permanent,Teaching Jobs,10000000,20131122T120000,20140220T120000,jobs.guardian.co.uk,Failed
48544,71875192,"DIGITAL PRODUCER FULL SERVICE INTEGRATED AGENCY,",Central London,FUTUREHEADS,full-time,permanent,Teaching Jobs,100000,20120805T120000,20121103T120000,jobs.guardian.co.uk,Failed
50585,72236157,CONTRACTS ACCOUNTANT,Eastern England,TIMOTHY JAMES CONSULTING,non-specified,contract,Accounting & Finance Jobs,100000,20130506T150000,20130705T150000,totaljobs.com,Failed


We found some high salary jobs, 100K for a full-time digital producer seems legit. However, 10,000,000 for a Technical Administrator seems way too much. We need to replace these two values in Id '69181134' and '71869302' with '0' so we can do the imputation

In [65]:
# Replace two targets with 0
df.loc[df.Id==69181134, 'Salary'] = '0'
df.loc[df.Id==71869302, 'Salary'] = '0'

In [66]:
# delet the test column
del df['test_salary']

After these steps, in 'Salary' column, we only have these types of the object:
- 12345
- 1234
- 1234.0
- 12345.0
- 123456
- 0

We then can move to the imputation of the Salary

In [67]:
# Change the data type tp float
df.Salary=df['Salary'].astype(float)

In [68]:
# Let us check some job title, for exapmle IT and Sales
df[df['Category'] == "IT Jobs"].describe()

,Id,Salary
count,1.434400e+04,14344.000000
mean,6.932804e+07,41389.858477
std,2.867071e+06,19769.704855
min,2.752705e+07,0.000000
25%,6.836150e+07,29000.000000
50%,6.917316e+07,40000.000000
75%,7.133585e+07,52500.000000
max,7.270524e+07,150000.000000


In [69]:
df[df['Category'] == "Sales Jobs"].describe()

,Id,Salary
count,5.349000e+03,5349.000000
mean,6.926131e+07,26898.953823
std,2.711321e+06,12031.293561
min,3.155936e+07,0.000000
25%,6.818042e+07,19000.000000
50%,6.905306e+07,26000.000000
75%,7.118655e+07,35000.000000
max,7.269594e+07,80000.000000


In [70]:
print (df.Category.value_counts())

IT Jobs                             14344
Healthcare & Nursing Jobs            8809
Engineering Jobs                     8210
Accounting & Finance Jobs            7138
Sales Jobs                           5349
Hospitality & Catering Jobs          4788
Teaching Jobs                        3779
PR, Advertising & Marketing Jobs     2752
Name: Category, dtype: int64


In [71]:
# Replace 0.0 with NaN
df.Salary.replace(0.0, np.nan,inplace=True)

In [72]:
# We implement the imputation base on the category
df["Salary"].fillna(df.groupby("Category")["Salary"].transform("mean"), inplace=True)
df.Salary.describe()

count     55169.000000
mean      34413.280982
std       15546.093046
min        5000.000000
25%       23296.000000
50%       31500.000000
75%       42500.000000
max      150000.000000
Name: Salary, dtype: float64

In [74]:
df.Salary=df["Salary"].round(decimals=1)

## Part6. Time column analysis
    We deal with the opendata and closedate columns

In [75]:
# make sure the 'T' in these two columns are in uppercase
df.OpenDate=df.OpenDate.str.upper()
df.CloseDate=df.CloseDate.str.upper()

In [76]:
# Build a regex pattern to filter these two columns
regex_time=re.compile(r'^201[0-5](02(0[1-9]|[12][0-9])|(0[469]|11)(0[1-9]|[12][0-9]|30)|(0[13578]|1[02])(0[1-9]|[12][0-9]|3[01]))T([0-1][0-9]|2[0-4])[0-5][0-9][0-5][0-9]$')

In [77]:
# Filter 'OpenData' to see is there any row that not follow the regex pattern
df[~(df['OpenDate'].str.match(regex_time))]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,Source
51062,72245549,COMPLIANCE TECHNICAL PROJECT CONSULTANT,South East London,MICHAEL PAGE FINANCIAL SERVICES,non-specified,permanent,Accounting & Finance Jobs,45000.0,20122302T120000,20120324T120000,totaljobs.com


In [78]:
# Filter 'CloseData'
df[~(df['CloseDate'].str.match(regex_time))]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,Source


Luckly, there is only one mistake in OpenDate column, it seems the month and date need to be exchanged

In [79]:
df.loc[df.Id==72245549, 'OpenDate'] = '20120223T120000'

In [80]:
df[~(df['OpenDate'].str.match(regex_time))]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,Source


In [81]:
# Then we convert the two columns to datatime format and do further study
df['OpenDateTest'] = pd.to_datetime(df['OpenDate'])
df['CloseDateTest'] = pd.to_datetime(df['CloseDate'])

In [82]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55169 entries, 0 to 55168
Data columns (total 13 columns):
Id               55169 non-null int64
Title            55169 non-null object
Location         55169 non-null object
Company          55169 non-null object
ContractType     55169 non-null object
ContractTime     55169 non-null object
Category         55169 non-null object
Salary           55169 non-null float64
OpenDate         55169 non-null object
CloseDate        55169 non-null object
Source           55169 non-null object
OpenDateTest     55169 non-null datetime64[ns]
CloseDateTest    55169 non-null datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(1), object(9)
memory usage: 5.5+ MB


In [83]:
# Is there any CloseDate 'Smaller' than the OpenData?
df.loc[df['CloseDateTest'] < df['OpenDateTest']]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,Source,OpenDateTest,CloseDateTest
4383,66190544,NEWLYQUALIFIED NIGHT NURSE,Mansfield,NON-SPECIFIED,non-specified,non-specified,Healthcare & Nursing Jobs,21000.0,20120226T150000,20120212T150000,careworx.co.uk,2012-02-26 15:00:00,2012-02-12 15:00:00
7469,67101212,TEMPORARY ASSISTANT FINANCIAL ACCOUNTANT,Cheshire,AXON RESOURCING LTD,full-time,contract,Accounting & Finance Jobs,47500.0,20131123T000000,20131024T000000,gaapweb.com,2013-11-23 00:00:00,2013-10-24 00:00:00
9321,67749488,CLINICAL ADVISOR BRISTOL,Bristol,HAYS HEALTHCARE,full-time,non-specified,Healthcare & Nursing Jobs,37000.0,20121004T150000,20120904T150000,staffnurse.com,2012-10-04 15:00:00,2012-09-04 15:00:00
22772,68844147,SOFT COMMODITIES RELATIONSHIP MANAGER,The City,TAYLOR JAMES RESOURCING LTD,non-specified,permanent,Accounting & Finance Jobs,35000.0,20120912T120000,20120614T120000,londonjobs.co.uk,2012-09-12 12:00:00,2012-06-14 12:00:00
24870,69038983,"SHAREPOINT CONSULTANT, WEST MIDLANDS, MICROSOF...",West Midlands,MORTIMER BELL INTERNATIONAL,non-specified,permanent,IT Jobs,47500.0,20120801T120000,20120702T120000,cwjobs.co.uk,2012-08-01 12:00:00,2012-07-02 12:00:00


In [84]:
# We then do the swap base on the condition
cond = df.CloseDateTest < df.OpenDateTest
df.loc[cond, ['OpenDate', 'CloseDate']] = df.loc[cond, ['CloseDate', 'OpenDate']].values

In [85]:
# Take a look, now we have correct Opendata and Closedate
df.loc[df['CloseDateTest'] < df['OpenDateTest']]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,Source,OpenDateTest,CloseDateTest
4383,66190544,NEWLYQUALIFIED NIGHT NURSE,Mansfield,NON-SPECIFIED,non-specified,non-specified,Healthcare & Nursing Jobs,21000.0,20120212T150000,20120226T150000,careworx.co.uk,2012-02-26 15:00:00,2012-02-12 15:00:00
7469,67101212,TEMPORARY ASSISTANT FINANCIAL ACCOUNTANT,Cheshire,AXON RESOURCING LTD,full-time,contract,Accounting & Finance Jobs,47500.0,20131024T000000,20131123T000000,gaapweb.com,2013-11-23 00:00:00,2013-10-24 00:00:00
9321,67749488,CLINICAL ADVISOR BRISTOL,Bristol,HAYS HEALTHCARE,full-time,non-specified,Healthcare & Nursing Jobs,37000.0,20120904T150000,20121004T150000,staffnurse.com,2012-10-04 15:00:00,2012-09-04 15:00:00
22772,68844147,SOFT COMMODITIES RELATIONSHIP MANAGER,The City,TAYLOR JAMES RESOURCING LTD,non-specified,permanent,Accounting & Finance Jobs,35000.0,20120614T120000,20120912T120000,londonjobs.co.uk,2012-09-12 12:00:00,2012-06-14 12:00:00
24870,69038983,"SHAREPOINT CONSULTANT, WEST MIDLANDS, MICROSOF...",West Midlands,MORTIMER BELL INTERNATIONAL,non-specified,permanent,IT Jobs,47500.0,20120702T120000,20120801T120000,cwjobs.co.uk,2012-08-01 12:00:00,2012-07-02 12:00:00


In [86]:
# Then we can delete two test columns
del df['OpenDateTest']
del df['CloseDateTest']

In [87]:
df.head(5)

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,Source
0,12612628,ENGINEERING SYSTEMS ANALYST,Dorking,GREGORY MARTIN INTERNATIONAL,non-specified,permanent,Engineering Jobs,25000.0,20130708T120000,20130906T120000,cv-library.co.uk
1,12612830,STRESS ENGINEER GLASGOW,Glasgow,GREGORY MARTIN INTERNATIONAL,non-specified,permanent,Engineering Jobs,30000.0,20120130T000000,20120330T000000,cv-library.co.uk
2,12612844,MODELLING AND SIMULATION ANALYST,Hampshire,GREGORY MARTIN INTERNATIONAL,non-specified,permanent,Engineering Jobs,30000.0,20121221T150000,20130120T150000,cv-library.co.uk
3,12613049,ENGINEERING SYSTEMS ANALYST / MATHEMATICAL MOD...,Surrey,GREGORY MARTIN INTERNATIONAL,non-specified,permanent,Engineering Jobs,27500.0,20131208T150000,20140206T150000,cv-library.co.uk
4,12613647,"PIONEER, MISER ENGINEERING SYSTEMS ANALYST",Surrey,GREGORY MARTIN INTERNATIONAL,non-specified,permanent,Engineering Jobs,25000.0,20130302T120000,20130501T120000,cv-library.co.uk


In [88]:
df.describe(include=['O'])

,Title,Location,Company,ContractType,ContractTime,Category,OpenDate,CloseDate,Source
count,55169,55169,55169,55169,55169,55169,55169,55169,55169
unique,53144,478,8486,3,3,8,2193,2418,106
top,PROJECT MANAGER,UK,NON-SPECIFIED,non-specified,permanent,IT Jobs,20120415T150000,20131208T000000,totaljobs.com
freq,60,8397,5389,41298,33637,14344,45,45,10102


In [89]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55169 entries, 0 to 55168
Data columns (total 11 columns):
Id              55169 non-null int64
Title           55169 non-null object
Location        55169 non-null object
Company         55169 non-null object
ContractType    55169 non-null object
ContractTime    55169 non-null object
Category        55169 non-null object
Salary          55169 non-null float64
OpenDate        55169 non-null object
CloseDate       55169 non-null object
Source          55169 non-null object
dtypes: float64(1), int64(1), object(9)
memory usage: 4.6+ MB


## Part7. Duplication checking

I drop the 'Source' column for finding any duplication. The reason behind this is one company may post certain job on different website with exactly the same information. Hence it should be considered as duplication when we handle the data set. Beside 'Source', we need to ignore 'Id' column too since it contains unique number and won't make any mean if we include it. 

In [90]:
# check duplication
cols = ['Title','Location','Company','ContractType','ContractTime','Salary','OpenDate','CloseDate']
duplicates = df[df.duplicated(cols,keep=False)]
print( "There are "+ str(len(duplicates)) + " duplicate records found")
duplicates.sort_values(by=duplicates.columns.tolist()).head(10)

There are 8 duplicate records found


,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,Source
8023,67290277,QUALITY ASSURANCE ENVIRONMENTAL MANAGER NOTTIN...,Nottingham,STEPHEN JAMES CONSULTING,non-specified,permanent,Healthcare & Nursing Jobs,35000.0,20120110T150000,20120409T150000,tntjobs.co.uk
34675,70086310,SENIOR FINANCIAL ADVISOR,London,FRAM EXECUTIVE SEARCH,non-specified,permanent,Accounting & Finance Jobs,40000.0,20130126T000000,20130225T000000,ifaonlinejobs.co.uk
47471,71808610,PENSIONS ADMINISTRATORS,UK,ABENEFIT2U,non-specified,contract,Accounting & Finance Jobs,24000.0,20130801T150000,20130831T150000,professionalpensionsjobs.com
54482,72653751,BARCLAYS FUTURE LEADERS DEVELOPMENT PROGRAMMES,Hackney,BARCLAYS,non-specified,non-specified,IT Jobs,36000.0,20121223T150000,20130221T150000,grb.uk.com
55165,72705205,PENSIONS ADMINISTRATORS,UK,ABENEFIT2U,non-specified,contract,Accounting & Finance Jobs,24000.0,20130801T150000,20130831T150000,cv-library.co.uk
55166,72705221,SENIOR FINANCIAL ADVISOR,London,FRAM EXECUTIVE SEARCH,non-specified,permanent,Accounting & Finance Jobs,40000.0,20130126T000000,20130225T000000,ifaonlinejobs.co.uk
55167,72705240,BARCLAYS FUTURE LEADERS DEVELOPMENT PROGRAMMES,Hackney,BARCLAYS,non-specified,non-specified,IT Jobs,36000.0,20121223T150000,20130221T150000,grb.uk.com
55168,72705244,QUALITY ASSURANCE ENVIRONMENTAL MANAGER NOTTIN...,Nottingham,STEPHEN JAMES CONSULTING,non-specified,permanent,Healthcare & Nursing Jobs,35000.0,20120110T150000,20120409T150000,tntjobs.co.uk


In [92]:
df.shape

(55169, 11)

In [93]:
df = df.drop_duplicates(cols, keep='last')

In [94]:
df.shape

(55165, 11)

In [95]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55165 entries, 0 to 55168
Data columns (total 11 columns):
Id              55165 non-null int64
Title           55165 non-null object
Location        55165 non-null object
Company         55165 non-null object
ContractType    55165 non-null object
ContractTime    55165 non-null object
Category        55165 non-null object
Salary          55165 non-null float64
OpenDate        55165 non-null object
CloseDate       55165 non-null object
Source          55165 non-null object
dtypes: float64(1), int64(1), object(9)
memory usage: 5.1+ MB


## Saving data
Save the cleaned data

In [96]:
#code to save output data
df.to_csv('dataset1_solution.csv',index=False)

## Summary
An error list about the original data set

- ID
    - All good
- Title
    - Redundancy of information, for example location and salary info
    - typo and misspelling
    - inappropriate use of special characters
    - inconsistency of word use
    - There is one row with only '*', we need replace it with non-specified
- Location
    - typo and misspelling, for example 'livepool' 
    - inconsistency of word use, such as '-' in between the locations
    - replace 'City' with 'The City'
- Company
    - NaN value, been replaced by 'NON-SPECIFIED'
    - typo and misspelling
    - inappropriate use of special characters
    - inconsistency of word use
- ContractType
    - NaN value, been replaced by 'non-specified'
    - rewrite the column values into 'full-time' and 'part-time'
- ContractTime
    - NaN value, been replaced by 'non-specified'
    - rewrote tje column values into 'permanent' and 'contract'
- Category
    - All good
- Salary
    - Nan, 0 or other characters that do not represent a vaild salary -> need to do the imputation
    - vaild salary with extra characters, include 'per year','annum','p/h','pa' and so on -> need to remove those characters then convert the number in annual salary
    - vaild salary in range and split by '-' and 'to'-> need to take the number in order to calculate the average salary
    - not vaild salary with 'k' in the end -> need to cut 'k' then times 1000 to get the annual salary
    - other suspicious numbers, for example a four digits number may repersent the monthly wage, need to be considered
    - two outliers that way beyond the normal salary
- OpenDate
    - one date invalid formation
    - some opendates 'later' than the closedates
- CloseDate
    - all good
- Source
    - find the Source that not end with .com, .uk, .net and .org -> change it to real website address
    
Plus there are some duplications, please check part7